In [2]:
# Dependencies
import requests
import pandas as pd
import json
import datetime
import requests
import calendar
import openpyxl
import seaborn as sns
from datetime import datetime, timedelta

from config import grant_type, client_id, client_secret, redirect_url,refresh_token

In [3]:
# Step 1: Create the OAuth link and authorize the access
oauth_link = f"https://hotels.cloudbeds.com/api/v1.1/oauth?client_id={client_id}&redirect_uri={redirect_url}&response_type=code"
print(f"Please go to this URL to authorize access: {oauth_link}")

# Step 2: Retrieve the authorization code from the redirect URL
auth_code = input("Enter the authorization code from the redirect URL: ")

url = 'https://hotels.cloudbeds.com/api/v1.1/access_token'
headers = {'Content-type': 'application/json'}

# Specify the grant type and other parameters based on your authentication flow
if grant_type == 'authorization_code':
    data = {
        'grant_type': 'authorization_code',
        'client_id': client_id,
        'client_secret': client_secret,
        'redirect_uri': redirect_url,
        'code': auth_code
    }
elif grant_type == 'refresh_token':
    data = {
        'grant_type': 'refresh_token',
        'client_id': client_id,
        'client_secret': client_secret,
        'refresh_token': refresh_token
    }

response = requests.post(url, data=data)

if response.ok:
    access_token_data = response.json()
    access_token = access_token_data["access_token"]
    token_type = access_token_data["token_type"]
    expires_in = access_token_data["expires_in"]
    refresh_token = access_token_data.get("refresh_token")
    print(f"Access token: {access_token}")
    print(f"Token type: {token_type}")
    print(f"Expires in: {expires_in} seconds")
    if refresh_token:
        print(f"Refresh token: {refresh_token}")
else:
    print("Error retrieving access token.")
    print(response.json())

Please go to this URL to authorize access: https://hotels.cloudbeds.com/api/v1.1/oauth?client_id=live1_172779_NOXF9fcUKEH6zMahDiSuv4T0&redirect_uri=https://webhook.site/2004f68e-9121-4331-a917-8bb8f74fbbb2&response_type=code
Access token: eyJraWQiOiI5eFUtV09RSWFtWXdLM3JneHF0NG9wbWNEVXdXWlQ3QXRUQm9TTzVuQ3hnIiwiYWxnIjoiUlMyNTYifQ.eyJ2ZXIiOjEsImp0aSI6IkFULkJFVTF1SjNJdURZeVk1d3p2N1NxMENmSmRIWTJ1RV9LVDJvX09haWNBdEUub2FyMTJlZ3d0d09aOHhJZmI1ZDciLCJpc3MiOiJodHRwczovL2lkcC5jbG91ZGJlZHMuY29tL29hdXRoMi9hdXNkNWcydTY5QmxKNFdBYzVkNiIsImF1ZCI6Imh0dHBzOi8vaG90ZWxzLmNsb3VkYmVkcy5jb20vYXBpIiwic3ViIjoia2VudHJvdGgrMUBnbWFpbC5jb20iLCJpYXQiOjE2ODY4NzI5NjQsImV4cCI6MTY4Njg3NjU2NCwiY2lkIjoibGl2ZTFfMTcyNzc5X05PWEY5ZmNVS0VINnpNYWhEaVN1djRUMCIsInVpZCI6IjAwdTFua2FwdGVSNjA2Z2ZYNWQ3Iiwic2NwIjpbIm9mZmxpbmVfYWNjZXNzIl0sImF1dGhfdGltZSI6MTY4NDE4NTk3OCwiYXNzb2NpYXRpb25JZHMiOltdLCJpc2xhbmQiOjEsInByb3BlcnR5SWRzIjpbMTcyNzc5XSwibWZkVXNlcklkIjoyMzMyOTIsInR5cGUiOiJwcm9wZXJ0eSJ9.W4sspdQCE0CXw2_FgzdsNv2-gFUhwX9JvABgB4wWEZUQ1aAqr

In [4]:
#Pull 1 year past data

url = "https://hotels.cloudbeds.com/api/v1.1/getReservationsWithRateDetails"

# Set up pagination parameters
page_number = 1
page_size = 100

results_past=[]

# Set the date range for the previous year
today = datetime.now()
one_year_ago = today - timedelta(days=365)

while True:
    # Parameters for the API request
    headers = {
        'Authorization': f'Bearer {access_token}'
    }
    params = {
        "reservationCheckOutFrom": one_year_ago.strftime("%Y-%m-%d"),
        "reservationCheckOutTo": today.strftime("%Y-%m-%d"),
        "sortByRecent": True,
        "pageNumber": page_number,
        "pageSize": page_size
    }

    # Make the API request
    response = requests.get(url, params=params, headers=headers)

    # Extract the 'data' field from the API response
    api_data = response.json().get('data', [])

    # Create a DataFrame from the extracted data
    df = pd.DataFrame(api_data)

    # Append the DataFrame to the overall results list
    results_past.append(df)

    # Check if there are more results
    if len(df) < page_size:
        break

    # Increment the page number for the next request
    page_number += 1

# Concatenate all the DataFrames into one
df_all_past = pd.concat(results_past, ignore_index=True)

# Print the final DataFrame
print(df_all_past)

      reservationID  isDeleted          dateCreated       dateCreatedUTC  \
0     8356837749213      False  2023-06-03 15:10:34  2023-06-03 20:10:34   
1     6412008537606      False  2023-06-01 22:01:36  2023-06-02 03:01:36   
2     8438510931843      False  2022-08-20 22:55:12  2022-08-21 03:55:12   
3     6334374831709      False  2023-06-09 18:16:00  2023-06-09 23:16:00   
4     5537693216724      False  2023-06-04 18:29:00  2023-06-04 23:29:00   
...             ...        ...                  ...                  ...   
7474  2981326909508      False  2022-01-27 08:46:12  2022-01-27 14:46:12   
7475  1723741176202      False  2022-01-21 07:48:29  2022-01-21 13:48:29   
7476  2625454162721      False  2022-01-16 09:11:45  2022-01-16 15:11:45   
7477  3118431497232      False  2021-12-05 16:11:26  2021-12-05 22:11:26   
7478  6597804802670      False  2022-01-07 01:46:26  2022-01-07 07:46:26   

             dateModified      dateModifiedUTC       status  \
0     2023-06-15 15:22:1

In [5]:
#Pull 1 year future data

url = "https://hotels.cloudbeds.com/api/v1.1/getReservationsWithRateDetails"

# Set up pagination parameters
page_number = 1
page_size = 100

results_future=[]

# Set the date range for the previous year
today = datetime.now()
one_year_ahead = today + timedelta(days=365)

while True:
    # Parameters for the API request
    headers = {
        'Authorization': f'Bearer {access_token}'
    }
    params = {
        "reservationCheckOutFrom": today.strftime("%Y-%m-%d"),
        "reservationCheckOutTo": one_year_ahead.strftime("%Y-%m-%d"),
        "sortByRecent": True,
        "pageNumber": page_number,
        "pageSize": page_size
    }


    # Make the API request
    response = requests.get(url, params=params, headers=headers)

    # Extract the 'data' field from the API response
    api_data = response.json().get('data', [])

    # Create a DataFrame from the extracted data
    df = pd.DataFrame(api_data)

    # Append the DataFrame to the overall results list
    results_future.append(df)

    # Check if there are more results
    if len(df) < page_size:
        break

    # Increment the page number for the next request
    page_number += 1

# Concatenate all the DataFrames into one
df_all_future = pd.concat(results_future, ignore_index=True)

# Print the final DataFrame
print(df_all_future)

     reservationID  isDeleted          dateCreated       dateCreatedUTC  \
0    3353380714939      False  2023-06-15 18:31:52  2023-06-15 23:31:52   
1    5231287282230      False  2023-06-15 18:25:19  2023-06-15 23:25:19   
2    6830989290835      False  2023-05-22 15:40:49  2023-05-22 20:40:49   
3    3849524585702      False  2023-06-15 18:01:04  2023-06-15 23:01:04   
4    7673753603249      False  2023-05-15 15:49:12  2023-05-15 20:49:12   
..             ...        ...                  ...                  ...   
584  9049776267868      False  2022-10-17 13:17:57  2022-10-17 18:17:57   
585  4480999257826      False  2022-10-04 13:52:19  2022-10-04 18:52:19   
586  8668732599792      False  2022-10-02 01:01:18  2022-10-02 06:01:18   
587  8901269886916      False  2022-09-21 12:43:29  2022-09-21 17:43:29   
588  9768135512101      False  2022-09-14 09:29:29  2022-09-14 14:29:29   

            dateModified      dateModifiedUTC      status reservationCheckIn  \
0    2023-06-15 18:

In [6]:
print(f"We received {len(df_all_past)} past reservations.")
print(f"We received {len(df_all_future)} future resservations.")

We received 7479 past reservations.
We received 589 future resservations.


In [7]:
#create cleaning data function
def expand_dataframe(df):
    expanded_rows = []
    
    for index, row in df.iterrows():
        detailed_rates = row['detailedRates']

        if detailed_rates:
            for date, price in detailed_rates.items():
                new_row = row.copy()
                new_row['date'] = date
                new_row['price'] = price
                expanded_rows.append(new_row)

    expanded_df = pd.DataFrame(expanded_rows)
    expanded_df.drop('detailedRates', axis=1, inplace=True)
    
    expanded_df['source'] = expanded_df['source'].apply(lambda x: x['name'])
    expanded_df.rename(columns={'source': 'source_name'}, inplace=True)
    
    expanded_df['adults'] = expanded_df['rooms'].apply(lambda x: x[0]['adults'])
    expanded_df.drop('rooms', axis=1, inplace=True)
    
    expanded_df['date'] = pd.to_datetime(expanded_df['date'])
    expanded_df['day'] = expanded_df['date'].dt.day
    expanded_df['month'] = expanded_df['date'].dt.month
    expanded_df['year'] = expanded_df['date'].dt.year
    
    return expanded_df

#create filter data for 'canceled' function
def filter_canceled_reservations(df):
    return df[df['status'] != 'canceled']

# Filter 'canceled' reservations
df_all_past = filter_canceled_reservations(df_all_past)
df_all_future = filter_canceled_reservations(df_all_future)

# Clean and expand dataframes
expanded_df_past = expand_dataframe(df_all_past[['reservationID', 'source', 'detailedRates', 'rooms']])
expanded_df_future = expand_dataframe(df_all_future[['reservationID', 'source', 'detailedRates', 'rooms']])

In [8]:
expanded_df_future.head()

,reservationID,source_name,date,price,adults,day,month,year
0,3353380714939,Booking.com,2023-06-15,41.0,1,15,6,2023
1,5231287282230,Hostelworld,2023-06-20,73.1,1,20,6,2023
1,5231287282230,Hostelworld,2023-06-21,73.1,1,21,6,2023
2,6830989290835,Hostelworld,2023-06-15,74.8,1,15,6,2023
2,6830989290835,Hostelworld,2023-06-16,93.5,1,16,6,2023


In [9]:
expanded_df_past.head()

,reservationID,source_name,date,price,adults,day,month,year
0,8356837749213,Website/Booking Engine,2023-06-10,52.0,1,10,6,2023
0,8356837749213,Website/Booking Engine,2023-06-11,41.0,1,11,6,2023
0,8356837749213,Website/Booking Engine,2023-06-12,39.0,1,12,6,2023
0,8356837749213,Website/Booking Engine,2023-06-13,39.0,1,13,6,2023
0,8356837749213,Website/Booking Engine,2023-06-14,39.0,1,14,6,2023


In [14]:
# Drop blank values
expanded_df_past.dropna(subset=['adults'], inplace=True)
expanded_df_future.dropna(subset=['adults'], inplace=True)

# Remove values after today's date for past_rev
expanded_df_past = expanded_df_past[expanded_df_past['date'] <= today]

# Remove values before today's date for future_rev
expanded_df_future = expanded_df_future[expanded_df_future['date'] >= today]

# Create a pivot table with sum of prices
pivot_table_past_rev = pd.pivot_table(expanded_df_past, values='price', index='month', columns='day', aggfunc='sum')
pivot_table_future_rev = pd.pivot_table(expanded_df_future, values='price', index='month', columns='day', aggfunc='sum')

# Reindex the columns to have labels 1-31
pivot_table_past_rev = pivot_table_past_rev.reindex(columns=range(1, 32))
pivot_table_future_rev = pivot_table_future_rev.reindex(columns=range(1, 32))

# Reindex the rows to have labels as month names and sort by year
sorted_months_past_rev = sorted(pivot_table_past_rev.index.unique(), key=lambda x: (expanded_df_past[expanded_df_past['month'] == x]['year'].min(), x))
pivot_table_past_rev = pivot_table_past_rev.reindex(index=sorted_months_past_rev)
pivot_table_past_rev.index = pivot_table_past_rev.index.map(lambda x: calendar.month_abbr[x])

sorted_months_future_rev = sorted(pivot_table_future_rev.index.unique(), key=lambda x: (expanded_df_future[expanded_df_future['month'] == x]['year'].min(), x))
pivot_table_future_rev = pivot_table_future_rev.reindex(index=sorted_months_future_rev)
pivot_table_future_rev.index = pivot_table_future_rev.index.map(lambda x: calendar.month_abbr[x])

# Fill blank cells with 0 in the DataFrames
pivot_table_past_rev.fillna(0, inplace=True)
pivot_table_future_rev.fillna(0, inplace=True)

# Export DataFrames to separate Excel files
pivot_table_past_rev.to_excel('Output/past_rev.xlsx', index=True, header=True, engine='openpyxl')
pivot_table_future_rev.to_excel('Output/future_rev.xlsx', index=True, header=True, engine='openpyxl')


In [16]:
# Convert 'adults' column to numeric values
expanded_df_past['adults'] = pd.to_numeric(expanded_df_past['adults'])
expanded_df_future['adults'] = pd.to_numeric(expanded_df_future['adults'])

# Create a pivot table with sum of adults per day divided by 64
pivot_table_past_occ = pd.pivot_table(expanded_df_past, values='adults', index='month', columns='day', aggfunc='sum') / 64
pivot_table_future_occ = pd.pivot_table(expanded_df_future, values='adults', index='month', columns='day', aggfunc='sum') / 64

# Reindex the columns to have labels 1-31
pivot_table_past_occ = pivot_table_past_occ.reindex(columns=range(1, 32))
pivot_table_future_occ = pivot_table_future_occ.reindex(columns=range(1, 32))

# Reindex the rows to have labels as month names and sort by year
sorted_months_past_occ = sorted(pivot_table_past_occ.index.unique(), key=lambda x: (expanded_df_past[expanded_df_past['month'] == x]['year'].min(), x))
pivot_table_past_occ = pivot_table_past_occ.reindex(index=sorted_months_past_occ)
pivot_table_past_occ.index = pivot_table_past_occ.index.map(lambda x: calendar.month_abbr[x])

sorted_months_future_occ = sorted(pivot_table_future_occ.index.unique(), key=lambda x: (expanded_df_future[expanded_df_future['month'] == x]['year'].min(), x))
pivot_table_future_occ = pivot_table_future_occ.reindex(index=sorted_months_future_occ)
pivot_table_future_occ.index = pivot_table_future_occ.index.map(lambda x: calendar.month_abbr[x])

# Fill blank cells with 0 in the DataFrames
pivot_table_past_occ.fillna(0, inplace=True)
pivot_table_future_occ.fillna(0, inplace=True)

# Export DataFrames to separate Excel files
pivot_table_past_occ.to_excel('Output/past_occ.xlsx', index=True, header=True, engine='openpyxl')
pivot_table_future_occ.to_excel('Output/future_occ.xlsx', index=True, header=True, engine='openpyxl')


In [23]:
# Calculate ADR for past_rev
pivot_table_past_adr = pivot_table_past_rev / 64 / (pivot_table_past_occ)

# Calculate ADR for future_rev
pivot_table_future_adr = pivot_table_future_rev / 64 / (pivot_table_future_occ)

# Fill blank cells with 0 in the DataFrames
pivot_table_past_adr.fillna(0, inplace=True)
pivot_table_future_adr.fillna(0, inplace=True)

# Export styled DataFrames to separate Excel files
pivot_table_past_adr.to_excel('Output/past_adr.xlsx', index=True, header=True, engine='openpyxl')
pivot_table_future_adr.to_excel('Output/future_adr.xlsx', index=True, header=True, engine='openpyxl')


In [24]:
# Calculate RevPAB for past_rev
pivot_table_past_revpab = pivot_table_past_rev / 64 * pivot_table_past_occ

# Calculate RevPAB for future_rev
pivot_table_future_revpab = pivot_table_future_rev / 64 * pivot_table_future_occ

# Fill blank cells with 0 in the DataFrames
pivot_table_past_revpab.fillna(0, inplace=True)
pivot_table_future_revpab.fillna(0, inplace=True)

# Export styled DataFrames to separate Excel files
pivot_table_past_revpab.to_excel('Output/past_revpab.xlsx', index=True, header=True, engine='openpyxl')
pivot_table_future_revpab.to_excel('Output/future_revpab.xlsx', index=True, header=True, engine='openpyxl')


In [30]:
# pivot_table_past_rev.to_csv('Output/past_rev.csv', index=True)
# pivot_table_future_rev.to_csv('Output/future_rev.csv', index=True)
# pivot_table_past_occ.to_csv('Output/past_occ.csv', index=True)
# pivot_table_future_occ.to_csv('Output/future_occ.csv', index=True)
